# Grid Search ile ARIMA Modellerindeki Katsayıların Elektrik Verisi Üzerine Uygulanması

Bu çalışmada EPİAŞ şeffaflık platformunda bulunan verilerin Grid Search ile ARIMA modellerindeki katsayıların hesaplanması hedeflenmektedir. Çekilen tüm veriler saatliktir. Çekilen veriler aşağıdaki tarih aralığındadırlar.

* Başlangıç Tarihi : 01 Ocak 2018
* Bitiş Tarihi : 30 Haziran 2024

In [12]:
import pandas as pd
import requests as rq
import json
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from matplotlib import pyplot
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

Kütüphaneler yüklendi. Sonraki bölümlerde kullanılacak olan fourier dönüşümü için Python'un içinde olan hazır komut kullanıldı. Bu komut fast fourier transfor(fft) olarak adlandırılmaktadır.  

In [2]:
date1 = "2018-01-01T00:00:00+03:00"
date2 = "2020-12-31T00:00:00+03:00" #sontarihin tamamını alıyor
date3 = "2021-01-01T00:00:00+03:00" #sontarihin tamamını alıyor
date4 = "2023-12-31T00:00:00+03:00" #sontarihin tamamını alıyor
date5 = "2024-01-01T00:00:00+03:00" #sontarihin tamamını alıyor
date6 = "2024-06-30T00:00:00+03:00" #sontarihin tamamını alıyor

base_path = "https://seffaflik.epias.com.tr/electricity-service"
header={"Content-Type" : "application/json"}

Epiaş'ın şeffaflık veri setinde bulunan veriler API ile elde edildi. Bu veriler için kullanılan web servis 2024 yılı için yeniden yayınlanan servistir. Bu verilerin çekilmesi gerekli olan tarih aralıkları farklıdır ve bu sebeple ön tarihler belirlendi.

In [3]:
ptfparam1  = json.dumps({"startDate": date1, "endDate": date2,"page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})
ptfparam2  = json.dumps({"startDate": date3, "endDate": date4,"page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})
ptfparam3  = json.dumps({"startDate": date5, "endDate": date6,"page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})

ptf1  = rq.request("POST", base_path+"/v1/markets/dam/data/mcp",  data = ptfparam1, headers=header)
ptf2  = rq.request("POST", base_path+"/v1/markets/dam/data/mcp",  data = ptfparam2, headers=header)
ptf3  = rq.request("POST", base_path+"/v1/markets/dam/data/mcp",  data = ptfparam3, headers=header)

ptf1  = ptf1.json()
ptf2  = ptf2.json()
ptf3  = ptf3.json()

ptf1  = pd.DataFrame(ptf1["items"])
ptf2  = pd.DataFrame(ptf2["items"])
ptf3  = pd.DataFrame(ptf3["items"])

ptf   = pd.concat([ptf1, ptf2 ,ptf3],axis = 0)

Piyasa takas fiyatı, EPİAŞ tarafından saatlik olarak açıklanmaktadır ve bir gün sonrası için aynı gün saat 14:00 kesinleşmektedir. Bu veriler çekilirken en fazla 3 yıllık tarih aralığı verilebilmektedir. Bu sebeple 3 aşamada çekilmiştir. Ayrı ayrı çekilen 3 tablo en sonunda birleştirilmiştir.

In [4]:
conparam1  = json.dumps({"startDate": date1, "endDate": date2,"page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})
conparam2  = json.dumps({"startDate": date3, "endDate": date4,"page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})
conparam3  = json.dumps({"startDate": date5, "endDate": date6,"page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})

consumption1  = rq.request("POST", base_path+"/v1/consumption/data/realtime-consumption",  data = conparam1, headers=header)
consumption2  = rq.request("POST", base_path+"/v1/consumption/data/realtime-consumption",  data = conparam2, headers=header)
consumption3  = rq.request("POST", base_path+"/v1/consumption/data/realtime-consumption",  data = conparam3, headers=header)

consumption1  = consumption1.json()
consumption2  = consumption2.json()
consumption3  = consumption3.json()

consumption1  = pd.DataFrame(consumption1["items"])
consumption2  = pd.DataFrame(consumption2["items"])
consumption3  = pd.DataFrame(consumption3["items"])

consumption   = pd.concat([consumption1, consumption2 ,consumption3],axis = 0)

Yukarıdaki hücrede gerçek zamanlı eletrik tüketimi verisi elde edilmiştir. PTF tablolarında olduğuna benzer şekilde 3 aşamda çekilmiş ve nihai tablo birleştirmiştir. Bu veri saatlik olarak açıklanmaktadır ve tüm Türkiye yük durumunu göstermektedir.

In [5]:
proparam1  = json.dumps({"startDate": "2018-01-01T00:00:00+03:00", "endDate": "2018-12-31T00:00:00+03:00","page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})
proparam2  = json.dumps({"startDate": "2019-01-01T00:00:00+03:00", "endDate": "2019-12-31T00:00:00+03:00","page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})
proparam3  = json.dumps({"startDate": "2020-01-01T00:00:00+03:00", "endDate": "2020-12-31T00:00:00+03:00","page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})
proparam4  = json.dumps({"startDate": "2021-01-01T00:00:00+03:00", "endDate": "2021-12-31T00:00:00+03:00","page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})
proparam5  = json.dumps({"startDate": "2022-01-01T00:00:00+03:00", "endDate": "2022-12-31T00:00:00+03:00","page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})
proparam6  = json.dumps({"startDate": "2023-01-01T00:00:00+03:00", "endDate": "2023-12-31T00:00:00+03:00","page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})
proparam7  = json.dumps({"startDate": "2024-01-01T00:00:00+03:00", "endDate": date6 ,"page": {"number": "", "size": "", "total": "", "sort": {"field": "date",  "direction": "ASC"}}})

production1  = rq.request("POST", base_path+"/v1/generation/data/realtime-generation",  data = proparam1, headers=header)
production2  = rq.request("POST", base_path+"/v1/generation/data/realtime-generation",  data = proparam2, headers=header)
production3  = rq.request("POST", base_path+"/v1/generation/data/realtime-generation",  data = proparam3, headers=header)
production4  = rq.request("POST", base_path+"/v1/generation/data/realtime-generation",  data = proparam4, headers=header)
production5  = rq.request("POST", base_path+"/v1/generation/data/realtime-generation",  data = proparam5, headers=header)
production6  = rq.request("POST", base_path+"/v1/generation/data/realtime-generation",  data = proparam6, headers=header)
production7  = rq.request("POST", base_path+"/v1/generation/data/realtime-generation",  data = proparam7, headers=header)

production1 = production1.json()
production2 = production2.json()
production3 = production3.json()
production4 = production4.json()
production5 = production5.json()
production6 = production6.json()
production7 = production7.json()

production1 = pd.DataFrame(production1["items"])
production2 = pd.DataFrame(production2["items"])
production3 = pd.DataFrame(production3["items"])
production4 = pd.DataFrame(production4["items"])
production5 = pd.DataFrame(production5["items"])
production6 = pd.DataFrame(production6["items"])
production7 = pd.DataFrame(production7["items"])

production01   = pd.concat([production1, production2 ,production3,production4,production5,production6,production7],axis = 0)

Bu hücrede lisanslı elektrik üretim miktarı elde edilmiştir. Lisans, EPİAŞ tarafından lisans yönetmeliğinde belirtilen kurulu gücün üstünde olan piyasa katılımcılarına verilmektedir. Bu veriler senelik elde edilebilmektedir ve 7 aşamada çekilmiştir. Saatlik ve kaynak bazlı olarak veriler ayrılmıştır.

In [7]:
production11  = rq.request("POST", base_path+"/v1/renewables/data/unlicensed-generation-amount",  data = proparam1, headers=header)
production12  = rq.request("POST", base_path+"/v1/renewables/data/unlicensed-generation-amount",  data = proparam2, headers=header)
production13  = rq.request("POST", base_path+"/v1/renewables/data/unlicensed-generation-amount",  data = proparam3, headers=header)
production14  = rq.request("POST", base_path+"/v1/renewables/data/unlicensed-generation-amount",  data = proparam4, headers=header)
production15  = rq.request("POST", base_path+"/v1/renewables/data/unlicensed-generation-amount",  data = proparam5, headers=header)
production16  = rq.request("POST", base_path+"/v1/renewables/data/unlicensed-generation-amount",  data = proparam6, headers=header)
production17  = rq.request("POST", base_path+"/v1/renewables/data/unlicensed-generation-amount",  data = proparam7, headers=header)

production11 = production11.json()
production12 = production12.json()
production13 = production13.json()
production14 = production14.json()
production15 = production15.json()
production16 = production16.json()
production17 = production17.json()

production11 = pd.DataFrame(production11["items"])
production12 = pd.DataFrame(production12["items"])
production13 = pd.DataFrame(production13["items"])
production14 = pd.DataFrame(production14["items"])
production15 = pd.DataFrame(production15["items"])
production16 = pd.DataFrame(production16["items"])
production17 = pd.DataFrame(production17["items"])

production02   = pd.concat([production11, production12 ,production13,production14,production15,production16,production17],axis = 0)

Yularıdaki hücrede lisanssız üretim miktarı elde edilmiştir. Lisanssız santraller, kurulu gücü belli bir değerin altında olduğu için EPİAŞ tarafından lisans alamamış piyasa katılımcılarıdır. Lisanssız üretime göre santral sayısı fazladır ve santral başına düşen üretim azdır. Bu tablolarda lisanslı üretim tablolarına benzer olarak senelik çekilebilmektedir. 7 aşamada çekilmiştir. Saatlik ve kaynak bazlı olarak veriler ayrılmıştır.

In [8]:
ptf          = ptf 
consumption  = consumption.drop(columns=['date', 'time'])
production01 = production01.drop(columns=['date', 'hour'])
production02 = production02.drop(columns=['date', 'time'])

hourlydata = pd.concat([ptf.reset_index(), consumption.reset_index(), production01.reset_index(), production02.reset_index()], axis = 1 )

Elde edilen piyasa takas fiyatı, elektrik tüketimi, lisanslı ve lisanssız elektrik üretimi "hourlydata" isimli tablo ile birleştirilmiştir. Çekilen veri aralığı tüm tablolarda aynıdır. Aynı tarihler için saatlik verilerdir.

In [11]:
p_range = range(14*24)  
d_range = range(14*24)
q_range = range(14*24)
best_score, best_cfg = 0.05 , None

In [13]:
 def gridsearch(x):   
    for p in p_range:
        for d in d_range:
            for q in q_range:
                order = (p, d, q)
                try:
                    # ARIMA modelini oluştur ve uydur
                    model = ARIMA(x, order=order)
                    model_fit = model.fit()

                    # Modelin performansını değerlendir (örneğin, MSE kullanarak)
                    mape = mean_absolute_percentage_error(x, model_fit.fittedvalues)

                    # En iyi skoru ve yapılandırmayı güncelle
                    if mape < best_score:
                        best_score, best_cfg = mape, order

                    print(f'ARIMA{order} MSE={mse}')

                except:
                    continue
    print('En iyi Konfigürasyon: ARIMA%s MSE=%.3f' % (best_cfg, best_score))


In [ ]:
gridsearch(hourlydata['priceUsd'])